In [ ]:
%pip -q install git+https://github.com/huggingface/transformers
%pip -q install -U datasets sentence-transformers loralib sentencepiece bitsandbytes accelerate einops  huggingface_hub
%pip -q install -U pypdf chromadb PyPDF2 langchain FlagEmbedding InstructorEmbedding langchain-community

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

###HuggingFace Login

In [ ]:
from google.colab import userdata

userdata.get('Polyjuiceai')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                             use_auth_token = True,
                                             load_in_4bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

###Building Pipeline

In [ ]:
# To create a text generation pipeline
pipe = pipeline("text-generation", # specify the task for the pipeline
                model = model,
                tokenizer = tokenizer,
                torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=2048,
                temperature=0.1,
                top_p=0.6,
                repetition_penalty=1.15,
                max_new_tokens=1024,
                device_map = 'auto',
                do_sample = False,
                top_k = 50,
                num_return_sequences = 1, # only one text sequence should be return for each input
                eos_token_id = tokenizer.eos_token_id,
                pad_token_id = tokenizer.eos_token_id,
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


###Setting up Langchain to retrieve PDFs


In [ ]:
# Load and process a single text files
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

171

In [ ]:
# splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

992

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
embedding_model_name = "BAAI/bge-base-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###Chroma DB

utilized for the storage and retrieval of vector embeddings.


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever


In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

###LangChain




In [ ]:
B_TOKEN, E_TOKEN = "<s>", "</s>"
INSTRUCT = "### Instruction:\n"
QUESTION = "\n\n### question:\n"
RESPONSE = "\n\n### Response:\n"

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature."""


# Creates a complete prompt
def create_prompt(user_query, system_prompt=DEFAULT_SYSTEM_PROMPT):
  prompt_template = B_TOKEN + INSTRUCT + system_prompt + QUESTION + user_query + RESPONSE + E_TOKEN
  return prompt_template


In [ ]:
# diffrent system propmt
system_prompt = """You are a knowledgeable, helpful, respectful, and honest assistant specializing in medical expertise and artificial intelligence. Always provide thorough and accurate information, ensuring your answers are as detailed and helpful as possible while maintaining safety and ethical standards. Your answers should avoid harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure that your responses are socially unbiased, positive in nature, and supported by relevant knowledge or research. When appropriate, provide examples, explanations, and context to enhance understanding. Always answer using the context text provided. Your answers should only answer the question once and not have any text after the answer is done. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. \
The generated responses must be accurate, precise, coherent, and consistent with the questions. They should be brief and not too long. For more advanced questions, provide a very short background and a concise summary while giving enough details. The next criterion in double asterisks is very important to follow: **Do not exceed the maximum length of the response. The maximum length of the generated response is defined as 512 new tokens; try not to exceed this and give a complete answer within this limit. Ensure that sentences are complete and ideas are fully conveyed within this constraint. If the response is too long, ensure to summarize the key points effectively while maintaining clarity and completeness. Always prioritize brevity without sacrificing essential details.**
"""

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

### RetrievalQA


In [ ]:
# Create the template prompt
prompt_template = create_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )

chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses all of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# To trim a given text
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:]
  else:
    return output_text


# Removes occurrences of a substring from a text
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '</s>')
    final_response = remove_substring(final_response, '</s>')

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      print(f"{source.metadata['source']}")

    print()

    return final_response

In [ ]:
query = "what are Large Language Models?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/JMLR.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/2403.13600v1.pdf

 Great question! Large Language Models (LLMs) refer to artificial intelligence models designed to process and generate human-like text based on
input data. These models can learn patterns from vast amounts of text data and use that knowledge to perform various natural language tasks such as
translation, summarization, and text generation. Notably, recent advancements involve fine-tuning these models for specific domains like medicine or
education, improving their performance in those areas. Some well-known LLMs include BERT, RoBERTa, and T5. These models have been trained on massive
datasets, enabling them to understand long-context dependencies and generate coherent and contextually appropriate responses.


# Retrieving questions and generating responses

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  dfQ.dropna()
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


###General QA

In [ ]:
file_path = str(input("Enter the file path: "))

Enter the file path: /content/HR_QA.csv


In [ ]:
# General Questions

questions_file_path = file_path
Q_dictionary = create_question_dict(questions_file_path)

for k in Q_dictionary.keys():
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However,



Sources:
/content/papers/2403.15360v2.pdf
/content/papers/2403.15360v2.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/JMLR.pdf
/content/papers/JMLR.pdf
/content/papers/JMLR.pdf



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...



Sources:
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/euae110.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/Allergy - 2023 - González‐de‐Olano - Clinical impact of the TPSAB1 genotype in mast cell diseases  A REMA study in a cohort.pdf
/content/papers/Allergy - 2023 - González‐de‐Olano - Clinical impact of the TPSAB1 genotype in mast cell diseases  A REMA study in a cohort.pdf
/content/papers/Allergy - 2023 - González‐de‐Olano - Clinical impact of the TPSAB1 genotype in mast cell diseases  A REMA study in a cohort.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/s00399-024-01014-0.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/s00399-024-01014-0.pdf



Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf



In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: Can you explain the difference between supervised and unsupervised learning in the context of AI?
 response:  In supervised learning, an AI model learns from labeled data, which means each input example comes with a corresponding output label. This
relationship allows the model to predict outputs for new inputs based on the learned patterns from the labeled data. Examples include image
classification, where images are labeled with their respective classes, and regression problems, where continuous values are assigned to inputs.

On the other hand, unsupervised learning involves processing unlabeled data, meaning there's no explicit correspondence between inputs and outputs.
Instead, the model identifies inherent structures and relationships within the data, often through techniques like clustering, dimensionality
reduction, or density estimation. Unsupervised learning can reveal hidden patterns and features in complex datasets, making it useful for applications
like anomaly detecti

In [ ]:
json_file_name = 'QA_HRGeneral.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QA_HRGeneral.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Create HR Advanced Paper Related QA Dictionary

In [ ]:
file_path = str(input("Enter the file path: "))

Enter the file path: /content/Advanced_Medical_QA.csv


In [ ]:
# Advanced Medical Questions

questions_file_path = file_path
Q_dictionary = create_question_dict(questions_file_path)

for k in Q_dictionary.keys():
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However,



Sources:
/content/papers/s12276-024-01233-y.pdf
/content/papers/s12276-024-01233-y.pdf
/content/papers/s12276-024-01233-y.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/s12276-024-01233-y.pdf
/content/papers/s12276-024-01233-y.pdf
/content/papers/s12276-024-01233-y.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...



Sources:
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/JMLR.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




Sources:
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf



Sources:
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf
/content/papers/RAFT Adapting Language Model to Domain Specific RAG.pdf



In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How do enhancer-promoter interactions mediated by CTCF and cohesin contribute to transcriptional regulation, and what are the implications of CTCF depletion on TAD structure and gene expression as described in the paper?
 response: 2. Enhancer-promoter interactions mediated by CTCF and cohesin play crucial roles in transcriptional regulation. CTCF acts as a boundary factor,
forming insulated neighborhoods or topologically associating domains (TADs), thereby preventing enhancers from interacting with distal promoters
indiscriminately. Cohesin, on the other hand, facilitates loop extrusion, enabling enhancer-promoter loops that bring enhancers into proximity with
their target promoters, promoting transcriptional activation.

3. Depletion of CTCF leads to the loss of enhancer-promoter looping and reduced TAD insulation, resulting in local transcriptional dysregulation.
Specifically, upregulated genes tend to be located closer to enhancers, suggesting their activation by ectopic enhanc

In [ ]:
json_file_name = 'QA_dvancedMedicalAI.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QA_dvancedMedicalAI.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)